# Retrieve information of the variants affecting the positions 23x49(BS), 12x48 (IBS, chain B interaction), 6x29, 4x39 (IBS), and 3x50 (IBS, CM Ionic Lock)
We have previous analyzed the sensitive regions of GPCRs:
1. Binding Site: we have defined the residues using a dataset of human aminergic GPCR crystallyzed with ligand. Then, analysing which of the 46 GPCRs (we had complete information of the variants affecting those GPCRs coming from the Hauser aper) presented variants in each of the positions that form the BS we selected position 23x49, as 14/46 GPCRs presented a non-homolous variant there.
2. Conserved Motif: we have defined three CM using the positions of GPCRmd, NaBS, IL and RTS. Then again, we have analyzed which of the 46 GPCRs had non-homologous variants in those positions. The most interesting one was 3x50 (IL) with 17/46. It is also part of the IBS.
3. Intracellular Binding Site: defined with the class A human GPCRs crystallized wth intracellular ligand. Then, after the non-homolous variant analysis of the 46 GPCRs (apart from position 3x50 that was already selected) we took 12x48, that was sometimes interacting with the secondary chain of the GPCRs and is present in 15/46 (though there were other two positions with 15/46). And also 6x29 and 4x39.

Now, we need to search for those positions 12x48, 23x49, 3x50, 4x39, 6x29, the variants information as we will have to select the most interesting ones to simulate. We would base our selection in its frequency (related with conservation) in the population (or computing the impact scores of the Hauser paper Sift, Polyphen) and the possible relation with a disease. 

How to retrieve this information? First, as the GPCRdb (used by Houser) is not updated as we checked in retrieve_variants_info_GPCRdb.ipynb, we will gather the MV information of all the GPCRs from GnomAD (previously ExAC, used by Mariona, now is not available). We will use the variant information (allele frequency, count, number and position) and, on the other hand, we will also base the variant selection in the possible relation with a disease using DisGeNET.

In [1]:
path_to_project="/home/martalo/Documentos/TFM/GPCR_variants"

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scipy
!{sys.executable} -m pip install bioservices
!{sys.executable} -m pip install biopython
!{sys.executable} -m pip install xlrd

from common_functions import *

import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist, squareform
import urllib

import mdtraj as md
import itertools
config(viewer='webgl')
from create_csv import * # this file is from Code/create_csv.py
import requests
import os
from matplotlib.ticker import FormatStrFormatter

/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:29: UserWarning: As of HTMD 1.16 the default ACEMD version for all protocols has changed to version 3. If you want to use version 2 protocols change the _version argument in the protocols or add `config(acemdversion=2)` to the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)
/home/martalo/miniconda3/envs/my_new_env_name/lib/python3.6/site-packages/htmd/versionwarnings.py:33: UserWarning: As of HTMD 1.16 the default number of threads HTMD spawns for calculations is set to 1. You can enable parallelism at your own risk using `config(njobs=-2)` in the beginning of your scripts. To disable this warning run once: `from htmd import _disableWarnings; _disableWarnings('1.16');`
  , UserWarning)


ffevaluate module is in beta version

Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049

HTMD Documentation at: https://www.htmd.org/docs/latest/



2020-12-01 10:26:33,049 - binstar - INFO - Using Anaconda API: https://api.anaconda.org


New devel HTMD version (1.23.5 python[3.6,<3.7.0a0,3.7,<3.8.0a0]) is available. You are currently on (1.19).There are several methods to update:    - Create a new conda env. using `conda create -n htmd1.23.5 htmd=1.23.5 -c acellera -c psi4 -c conda-forge`    - Create a brand new conda installation and run `conda install htmd -c acellera -c psi4 -c conda-forge`    - Run: `conda update htmd -c acellera -c psi4 -c conda-forge` (NOT RECOMMENDED)



In [2]:
currentdir ="/home/martalo/Documentos/TFM/GPCR_variants/Code/potential_high_impact_variants/"
parentdir = "/home/martalo/Documentos/TFM/GPCR_variants/"
datapath="/home/martalo/Documentos/TFM/GPCR_variants/Data/studied_GPCR_vars"
resultspath="/home/martalo/Documentos/TFM/GPCR_variants/Results/studied_GPCR_vars"

## Functions 

In [26]:
def create_excel_from_raw(file_family_raw, file_family_csv):
    '''Read the txt file of the raw family information and build a ordered csv in the result 
    path'''
    # open the raw file 
    first_line=True
    with open(os.path.join(datapath,file_family_raw),"r") as infile:
        # build a csv file with the ordered info: family, name, short name, UniPort Id of each of the structures
        with open(os.path.join(resultspath,file_family_csv), 'w') as outfile:    
            mywriter = csv.writer(outfile,delimiter=';')
            for line in infile:
                if first_line:
                    mywriter.writerow(['Family','Name','Short','Uniprot ID'])
                    first_line=False
                if len(line) - len(line.lstrip())> 8 and not first_line:
                    el_li=line.strip().split('","')
                    fam=el_li[2]
                    short_nm=el_li[10]
                    long_nm=el_li[11]
                    uniprot=el_li[15]
                    mywriter.writerow([fam,long_nm,short_nm,uniprot])
    infile.close()
    outfile.close()
    return print('The file', file_family_csv,'has been succesfully created')

def search_struc_pdbs(original_csv, extended_csv): 
    '''With the information of the family, name, short name and Uniport ID, create
    another file adding the Uniprot entry name and the information of the structure
    from pdb (three colums, if there is active, intermediate and active structure)'''
    u = UniProt()# 'connection' with Uniprot services             
    with open(os.path.join(resultspath,original_csv),"r") as infile:
        with open(os.path.join(resultspath,extended_csv), 'w') as outfile:   
            mywriter = csv.writer(outfile,delimiter=';') 
            myreader = csv.reader(infile, delimiter=';')
            for row in myreader:
                if row[0]=="Family":
                    mywriter.writerow(row+["Uniprot entry","Struc"])
                    continue
                struccell=""
                entry=""
                uprot=row[3]
                print("\n\n",row[2])
                if uprot:
                    data=u.quick_search("id:%s" % uprot)
                    if data:
                        entry=data[uprot]['Entry name'].lower()
                        struc_res=obtain_struc_pdb(entry)
                        if struc_res:
                            struccell=struc_res
                            print(struccell)
                        else:
                            temp=requests.get('http://gpcrdb.org/services/structure/template/'+entry).json()
                            if temp:
                                temp_res=obtain_struc_pdb(temp)
                                if temp_res:
                                    struccell="[Model]: "+temp_res
                                    print(struccell)
                                else:
                                    print("-----No struc for template")
                            else:
                                print("-----No template")
                    else:
                        print("-----Uprot ID not found")
                else:
                    print("-----No uprot ID")
            
                mywriter.writerow(row+[entry,struccell])
    infile.close()
    outfile.close()
    return print('The file', extended_csv,'has been succesfully created')

def add_colum_names(csv_file_new, csv_file_old):
    '''Add colum names to the struc file and store it into new file'''
    with open(os.path.join(resultspath,csv_file_old),"r") as strucfile_old:
        with open(os.path.join(resultspath,csv_file_new),"w") as strucfile_new:
            myreader= csv.reader(strucfile_old, delimiter=';')
            mywriter= csv.writer(strucfile_new, delimiter=';') 
            for myrow in myreader:#iterate over the struc file rows
                if myrow[0]=="Familiy":
                    newrow=['Family', 'Name', 'Short', 'Uniprot ID','Uniprot entry','Struc','Struc1','Struc2']
                    print(newrow)
                    mywriter.writerow(newrow)
                else:
                    mywriter.writerow(myrow)
    strucfile_new.close()
    strucfile_old.close()
    return None

def create_csv_file(variants_excel, variants_csv):
    '''Tranform an Excel xlsx file into a csv file'''
    varFileAux=pd.read_excel(os.path.join(datapath,variants_excel))
    varFileAux.to_csv(os.path.join(datapath,variants_csv), sep=";", index=False)
    return None

def merge_struc_cols(csv_new, csv_old):
    '''Merged the possible three 'struc' pdb values, linked by '&' '''
    mynewrow=[]
    with open(os.path.join(resultspath,csv_old),"r") as file_old:
        with open(os.path.join(resultspath,csv_new),"w") as file_new:
            myreader= csv.reader(file_old, delimiter=';')
            mywriter= csv.writer(file_new, delimiter=';') 
            for myrow in myreader:#iterate over the file rows
                count=0
                if myrow[0]=="Family":
                    mywriter.writerow(myrow[:-2])
                    continue
                else: 
                    mynewrow=myrow
                    mynewrow[5]=myrow[5].replace(',', ' &')
                    mywriter.writerow(mynewrow)
                    mynewrow=[]
    file_new.close()
    file_old.close()
    return None

def get_gene(entry_name):
    '''Obtain the ENSEMBL id of the gene of the entry name'''
    uprot_map=uniprot_mapping('ACC+ID', 'ENSEMBL_ID', entry_name)#tab-delimited output of the mapping
    ens_id= uprot_map.decode().strip("\n").split("\t")[-1]
    return ens_id

def short_AA(AA_three_letters):
    '''Tranform 3 letter AA to one letter AA'''
    amin=AA_three_letters.upper()
    d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}
    return d[amin]



## Build GPCR 'variants' file: GPCRdb index, GnomAD variants info, DisGeNET.
It we be obtained with the index file from GPCRdb and the variant data from GnomAD plus DisgeNET.

#### 1. Download  the txt file with the family information
First, we need the file with the names of all the GPCRs (obtained from https://gpcrdb.org/mutational_landscape/statistics, 'proteins_and_families_rawALL.txt') to be an 'index'.

Build the first csv (/Results) that will contain the family, name, short name and Uniprot ID of all the GPCRs that are in GPCRdb (https://gpcrdb.org/mutational_landscape/statistics) from the raw file/seed file 'proteins_and_families_rawALL.txt':


In [27]:
# transform to csv the raw txt with the family/name/short name/Uniprot ID info 
#of all the GPCRs from GPCRdb

# Only call if the previous csv have changed
#create_excel_from_raw("proteins_and_families_rawALL.txt", 'myprot_list_ALL.csv')

###################NOTE##########################################################
# Whenever one of the created csv files is manually edited with Libre Office it somehow 
#changes the format, so to edit it use the plain text editor!


#### 2. Add the structure information in a new csv
Build another csv file (/Results) that stores the same info as before plus the  Uniprot entry name and three possible extra columns with the pdb structures names (active, intermediate, inactive):

In [28]:
# Only call if the previous csv have changed
#search_struc_pdbs("myprot_list_ALL.csv", 'myprot_list_struc_ALL_old.csv')

# correct the column names
#add_colum_names("myprot_list_struc_ALL.csv", "myprot_list_struc_ALL_old.csv")

# Put struc2 and struc3 in the same column linked by '&'
#merge_struc_cols("myprot_list_struc_ALL_merged.csv", "myprot_list_struc_ALL.csv")

#### 3. Build the final 'variants_ALL_GNOMAD.csv' 
It will contain the GPCR pdb structure info plus the info of the variants that comes from GnomAD. We will use gnomAD v2 data set which contains data from 125,748 exomes and 15,708 whole genomes from unrelated individuals sequenced as part of various disease-specific and population genetic studies, totalling 141,456 individuals, and is aligned against the GRCh37 reference sequence (the latest version v3 has less exomes information, recommended if you are interesting in non-coding regions, not our case).
https://gnomad.broadinstitute.org/

First, we have an example on how to make queries in GnomAD, as they do not have an API:


In [29]:
# Query GnomAD to get: hgvsp (to get segment position, original AA and variant AA), allele freq,
#num, count and annotation from an Ensembl ID
import pprint
prettyprint = pprint.PrettyPrinter(indent=2).pprint

def fetch(jsondata, url="https://gnomad.broadinstitute.org/api"):
    # The server gives a generic error message if the content type isn't
    # explicitly set
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=jsondata, headers=headers)
    json = response.json()
    if "errors" in json:
        raise Exception(str(json["errors"]))
    return json

def get_variant_list(gene_id, dataset="gnomad_r2_1"):
    # Note that this is GraphQL, not JSON.
    # Possible data to retrieve:
    # pos	rsid	ref	alt	consequence	exome	flags	lof	consequence_in_canonical_transcript
    #gene_symbol	hgvsc	lof_filter	lof_flags	hgvsp	reference_genome	variant_id	
    #genome_af	genome_ac	genome_an	genome_ac_hemi	genome_ac_hom	genome	exome_af	
    #exome_ac	exome_an	exome_ac_hemi	exome_ac_hom
    
    # We need the hgvsp (segNum, originalAA and variantAA), allele freq, allele num, allele 
    # count, consequence (get only MVs). We lack number of homozygotes genome_ac_hom
    fmt_graphql = """
    {
        gene(gene_id: "%s") {
          variants(dataset: %s) {
            hgvsp            
            genome {
                af
                ac
                an
            }
            consequence
            rsid
          }
        }
      }
    """# rsid, variant_id: variantId -> maybe useful
    
    # This part will be JSON encoded, but with the GraphQL part left as a
    # glob of text.
    req_variantlist = {
        "query": fmt_graphql % (gene_id, dataset),
        "variables": {}
        }
    response = fetch(req_variantlist)#json format
    return response["data"]["gene"]["variants"]
    #return response

# Examples of how the info is obtained, the results can be compared with the ones from
# gnomad_python_api.git in the output folder, file: variants.tsv
#prettyprint(get_variant_list("ENSG00000169174"))
# print(get_variant_list("ENSG00000135312"))
        

In [30]:
# retrieve MV data of the GPCR of the index file

def find_variants_gnomad(index_csv, final_csv):
    '''Use the info of the index file to look for the variants stored in GnomAD and store
    them in the final_csv'''
    
    non_identified=0 # keep track of the non-identified GPCRs
    not_entry=0
    with open(os.path.join(resultspath,index_csv),"r") as myprotfile:
    # Build new file merging structure info (index file) + variants info (GnomAD), containing:
    # Family	Name	Short	Uniprot ID	Uniprot entry	Struc	Ligandtype	gene
    #GPCRdb	SequenceNumber	Segment	GPCRdbWT	NMaa	MutationType	Allele Count
    #Allele Frequency	Allele Number	Number of Homozygotes	sift_word	sift_score
    #polyphen_word	polyphen_score	GProteinInteraction	ArrestinInteraction	
    #ActivationPathway	MicroSwitch	SodiumPocket	foldchangeMaxAbs	diseaseId	
    #score	diseaseName	Type	PTMsite	LB_structure	LB_fam

        with open(os.path.join(resultspath,final_csv), 'w') as outfile:   
            myprotread = csv.reader(myprotfile, delimiter=';') # csv with struc info (index)
            mywriter = csv.writer(outfile,delimiter=';') # final csv
            for myrow in myprotread:#iterate over the struc file rows
                if myrow[0]=="Family": # write first colum of the new csv
                    newrow=myrow+['Ligandtype','gene','varID', 'GPCRdb', 'SequenceNumber', 'Segment', 'GPCRdbWT', 'NMaa', 'MutationType', 'Allele Count', 'Allele Frequency', 'Allele Number', 'Number of Homozygotes', 'sift_word', 'sift_score', 'polyphen_word', 'polyphen_score', 'GProteinInteraction', 'ArrestinInteraction', 'ActivationPathway', 'MicroSwitch', 'SodiumPocket', 'foldchangeMaxAbs', 'diseaseId', 'score', 'diseaseName', 'Type', 'PTMsite', 'LB_structure', 'LB_fam']
                    mywriter.writerow(newrow)
                    # Containing the elements of struc file plus the interesting ones of the var 
                    continue
                entry=myrow[4] # colum 4 from csv with struc info: Uniprot entry
                print('Entry',entry)
                if not entry: # if there is not Uniprot entry
                    mywriter.writerow(myrow)
                    not_entry+=1
                    continue
                
                ens_id= get_gene(entry) # get Ensembl ID
                no_id=False
                try: # now check this Ensembl ID in gnomAD
                    gnom=get_variant_list(ens_id)  # list of dict of the variants found in
                #gnomAD of the entry
                except:
                    print('No Ensemble ID for ', entry)
                    no_id=True
                    non_identified+=1
                
                # if the search in GnomAD has been succesfull
                if not no_id:
#                     print('Esemble ID correct')
                    # get the structure data of the protein from GPCRdb
                    gpcr_pos_info=get_gpcr_pos_info(entry)
                    # keys as the 'sequence_number' (position) and values -> protein_segment, gnum

                    found=False                    
                    for gnom_var in gnom:#iterate over the found variants for the entry
                        if gnom_var["consequence"] =="missense_variant":
#                             print('MV in GnomAD')
                            found=True
                            
                            var_id=gnom_var['rsid']
                            
                            # get the seqNum, originalAA and variantAA from 'hgvsp' 
                            AAdata=gnom_var['hgvsp']
                            fromAA_long=AAdata[2:5] # Ex: Ala
                            toAA_long=AAdata[-3:] # Ex: Gly
                            fromAA=short_AA(fromAA_long) # Ex: A
                            toAA=short_AA(toAA_long) # Ex: G
                            seqNum=int(re.findall('\d+', gnom_var['hgvsp'] )[0])# get only the numbers
#                             print('seqNum:', seqNum)
                            
                            # get allele data
                            if gnom_var["genome"]:
                                gen_dict=gnom_var["genome"]
                                allele_count=gen_dict['ac']
                                allele_freq=gen_dict['af']
                                allele_num=gen_dict['an']
                            else:
                                allele_count='?'
                                allele_freq='?'
                                allele_num='?'
                            
                            #look for the gnum of the variant position
                            gpcrNum=""
                            try: 
                                this_pos_info=gpcr_pos_info[seqNum]
                                if this_pos_info["display_generic_number"]:
                                    gpcrNum=this_pos_info["display_generic_number"]
                                Segment=this_pos_info["protein_segment"]
                            except:
                                print('Position/SeqNum not found')                                
                                gpcrNum='?'
                                Segment='?'
                                
                            hom_count='?'###### Lack from GnomAD
                            Ligandtype="?"
                            MutationType=check_aa_type_changed(fromAA,toAA)
                            sift_word="?"
                            sift_score="?"
                            polyphen_word="?"
                            polyphen_score="?"
                            GProteinInteraction="?"
                            ArrestinInteraction="?"
                            ActivationPathway="?"
                            MicroSwitch="?"
                            SodiumPocket="?"
                            foldchangeMaxAbs="?"
                            diseaseId="?"
                            score="?"
                            diseaseName="?"
                            Type="?"
                            PTMsite="?"
                            LB_structure="?"
                            LB_fam="?"
                            
                            mywriter.writerow(myrow + [Ligandtype, ens_id, var_id, gpcrNum, seqNum, Segment, fromAA, toAA, MutationType, allele_count, allele_freq, allele_num, hom_count, sift_word, sift_score, polyphen_word, polyphen_score, GProteinInteraction, ArrestinInteraction, ActivationPathway, MicroSwitch, SodiumPocket, foldchangeMaxAbs, diseaseId, score, diseaseName, Type, PTMsite, LB_structure, LB_fam])
                    if not found:
                        print("No variants found for %s" % entry)
                        mywriter.writerow(myrow)
    print('Without ensembl ID:', non_identified)
    print('Without entry:', not_entry)
    return None

# find_variants_gnomad("myprot_list_struc_ALL_merged.csv","variants_ALL_GNOMAD.csv") 



## Filter the variants
#### 1. Filter the data with allele information and affecting the positions of interest


In [31]:
def filter_var_file(var_all, var_filtered, pos_to_filter):
    '''Filter the file with all the variants downloaded from GnomAD to only
    get the ones with allele fre, num, count and affecting the interesting positions'''
    with open(os.path.join(resultspath,var_all),"r") as myprotfile:
        with open(os.path.join(resultspath,var_filtered), 'w') as outfile:   
            myprotread = csv.reader(myprotfile, delimiter=';') # csv with struc info (index)
            mywriter = csv.writer(outfile,delimiter=';') # final csv
            for myrow in myprotread:#iterate over the struc file rows
                if myrow[0]=="Family": # write first colum of the new csv
                    mywriter.writerow(myrow)                
                    continue
                else:
                    try: # if the gnum is different than '?'
                        gnum=gnum_all_to_one("gpcrdb",myrow[9])
                    except:
                        gnum=False
                        continue

                    allele=[myrow[15], myrow[16], myrow[17]] # allele freq, num, count            
                    info_ok= True
                    for element in allele:
                        if element == '?':
                            info_ok=False
                    if info_ok and gnum in pos_to_filter:
                        mynewrow=myrow
                        mynewrow[9]=gnum
                        
                        #DisGeNET info
                        disCols=[myrow[29],myrow[30],myrow[31]]
                        varIDrs=myrow[8]
                        
                        
                        mywriter.writerow(mynewrow)

    return (print("The file",var_filtered, "has been succesfully created."))

interest_pos=['23x49','12x48','4x39','6x29', '3x50'] # BS, IBS (3), CM-IL (also of IBS)

# filter_var_file("variants_ALL_GNOMAD.csv", "variants_ALL_GNOMAD_filtered.csv", interest_pos)

## Retrieve information from DisGeNET
Now, we want to get the information of DisGeNEt to fill in the information of columns: 'diseaseId', 'score', 'diseaseName' of the previous file. To do so, we will use the rest API disgenet.org/api/, specially, the Variant-Disease Associations (VDAs). The VDAs service provides an interface for accessing variant-disease associations from DisGeNET. The VDAs can be retrieved for a specific variant (ex: rs121909213), or list of variants. It is recommended that the list of query variants is not longer than 100 entities. The service returns the results formated in TSV, JSON, or XML.

In [32]:
def request_disgenet(varID):
    '''Get variant relation with a disease from DisgeNet using the variant id (Ex:rs121909211)'''
    # https://www.disgenet.org/api/#/VDA/vdaByVariant
    base = 'https://www.disgenet.org/api'
    tool = 'vda/variant'
    url = base+'/'+tool+'/'+varID
    response =  urllib.request.urlopen(url)
    return (response.read())

def get_disease_info(req_response):
    '''Select only the three important values of the DisgeNET response for each variant,
    Id, disease name and score'''
    # Available information:
# {"variantid":"rs121909211","gene_symbol":"TGFBI","variant_dsi":"0.724","variant_dpi":"0.2",
#  "variant_consequence_type":"missense variant","diseaseid":"C1275685",
#  "disease_name":"Avellino corneal dystrophy","disease_class":"C16;C11",
#  "disease_class_name":"   Congenital, Hereditary, and Neonatal Diseases and Abnormalities;    
#  Eye Diseases","disease_type":"disease","disease_semantic_type":"Disease or Syndrome",
#  "score":0.9,"ei":1.0,"year_initial":1998,"year_final":2019,"source":"ALL"}

    req_list = json.loads(req_response)
    list_diseases=[]
    dis_dic={}
    print('Elements of the returned list of related diseases:')
    for element in req_list:
        dis_dic['disease_id']=element['diseaseid']
        dis_dic['score']=element['score']
        dis_dic['diseaseName']=element['disease_name']
        list_diseases.append(dis_dic)
        print(dis_dic)
        dis_dic={}
    return list_diseases


def major_disease(list_of_diseases):
    '''Returns a tupple with the dictionary of the disease with the higest score and 
    the total number of related diseases'''
    mvp=list_of_diseases[0] # most important disease
    number_dis=len(list_of_diseases) # number of related diseases
    for element in list_of_diseases:
        if element['score'] > mvp['score'] :
            mvp=element
    print('The disease with higher score is:',mvp)
    print('Number of related diseases:',number_dis)
    return(mvp, number_dis)

# Examples:
# related_diseases= get_disease_info(request_disgenet('rs121909211')) #list of diseases
# most_imp_dis=major_disease(related_diseases)
# print(most_imp_dis[1])

As each varID may be related with more than one disease, we will store the complete information of the related disease with highest score but also the total number of related diseases for each variant. 

In [33]:
#disCols=[29,30,31]  'diseaseId'-> 'diseaseid', 'score' -> 'score' (Score VDA: Variant-Disease 
# Asociation Score),'diseaseName' -> 'disease_name'

def add_dis_info(var_filtered, var_disgenet):
    '''Add the information of DisGeNET, id, socre, name, and the number of related
    diseases for each varID'''
    with open(os.path.join(resultspath,var_filtered),"r") as myprotfile:
        with open(os.path.join(resultspath,var_disgenet), 'w') as outfile:   
            myprotread = csv.reader(myprotfile, delimiter=';') # csv with struc info (index)
            mywriter = csv.writer(outfile,delimiter=';') # final csv
            for myrow in myprotread:#iterate over the struc file rows
                if myrow[0]=="Family": # write first colum of the new csv
                    mynewrow=myrow
                    mynewrow.append("related_diseases")
                    mywriter.writerow(mynewrow)                
                    continue
                else:
                    varID=myrow[8]
                    try:
                        related_dis= get_disease_info(request_disgenet(varID)) #list of diseases
                    except:
                        print('No matches in DisGeNET for:', varID)
                        mywriter.writerow(myrow)
                        continue
                    imp_info=major_disease(related_dis)
                    most_imp_dis=imp_info[0]# disease info with the highest score
                    num_dis=imp_info[1] # number of related diseases
                    
                    mynewrow=myrow
                    mynewrow[29]=most_imp_dis['disease_id']
                    mynewrow[30]=most_imp_dis['score']
                    mynewrow[31]=most_imp_dis['diseaseName']
                    mynewrow.append(num_dis)
                mywriter.writerow(mynewrow) 
    myprotfile.close()
    outfile.close()
    return (print('The file', var_disgenet,'has been succesfully created'))
                      
# add_dis_info("variants_ALL_GNOMAD_filtered.csv", "variants_ALL_GNOMAD_DISGENET.csv")

The resulting file 'variants_ALL_GNOMAD_DISGENET.csv' in the 'Results' file contains all the info of the variants affenting all kind GPCRs that had information of the allele freq, number and count in GnomAd. In addition, the variants that have information on their relation with diseases stored in DisGeNET is also added. 

## Compute the Impact Scores: SIFT, PolyPhen
#### 1. SIFT: Sorting Intolerant From Tolerant
It predicts whether an amino acid substitution is likely to affect protein function based on sequence homology and the physico-chemical similarity between the alternate amino acids. The data that Ensembl provides for each amino acid substitution is a score and a qualitative prediction (either 'tolerated' or 'deleterious'). The score is the normalized probability that the amino acid change is tolerated so scores nearer zero are more likely to be deleterious. The qualitative prediction is derived from this score such that substitutions with a score < 0.05 are called 'deleterious' and all others are called 'tolerated'. 

#### 2. PolyPhen: 
It predicts the effect of an amino acid substitution on the structure and function of a protein using sequence homology, Pfam annotations, 3D structures from PDB where available, and a number of other databases and tools (including DSSP, ncoils etc.). As with SIFT, for each amino acid substitution where Ensembl has been able to calculate a prediction, it provides both a qualitative prediction (one of 'probably damaging', 'possibly damaging', 'benign' or 'unknown') and a score. The PolyPhen score represents the probability that a substitution is damaging, so values nearer one are more confidently predicted to be deleterious (note that this the opposite to SIFT). The qualitative prediction is based on the False Positive Rate of the classifier model used to make the predictions. 

In the case of SIFT we would need to use SIFT dbSNP that provides SIFT predictions for a list nonsynonymous SNPs (rsIDs) from NCBI's dbSNP database. But in order to use an API we will use the API rest form Ensembl VEP (Variant Effect Predictor) that will directly give us the SIFT and PolyPhen information we need.

https://rest.ensembl.org/documentation/info/vep_id_post
https://sift.bii.a-star.edu.sg/www/SIFT_dbSNP.html

In [34]:

def request_VEP(varID):
    '''Get variant SIFT and PolyPhen2 from VEP (Ensembl) using the variant id (Ex:rs121909211)'''
    server = "https://rest.ensembl.org"
    ext = "/vep/human/id/"+varID+"?"

    r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    decoded = r.json()
#     print(decoded)
    return(decoded)

# Examples:
# response=request_VEP("rs56116432")
# response=request_VEP("rs199805893")
# response=request_VEP("rs759659059")

def filter_VEP_response(VEPresponse,ensID,varID):
    '''Filer the respose, list type, from VEP, after quering a varID. Get the 
    'sift_prediction','sift_score','polyphen_prediction','polyphen_score' for 
    each transcriptID (after checking is the correct EnsemblID)'''
#     pred_dic={}
    keys=['sift_prediction','sift_score','polyphen_prediction','polyphen_score']
    for element in VEPresponse:
        prediction_list=element['transcript_consequences']
        pred_dic={}
        for dictionary in prediction_list:
            predictions=True
#             print(dictionary)
#             print(dictionary['gene_id'], ensID)
            if dictionary['gene_id']==ensID:# check ensemblID
#                 print('ensID ok')
                transID=dictionary['transcript_id'] # store each transcriptID
#                 print('transID')
                for value in keys:# check all the predictions exist
                    if value not in dictionary.keys():
                        predictions=False  
#                         print('Not all the keys in the dictionary')
                    if predictions:
                        pred_dic[transID]={'Sift': [ dictionary['sift_score'], dictionary['sift_prediction'] ],
                                            'Polyphen':[dictionary['polyphen_score'], dictionary['polyphen_prediction'] ]}
            else:
                print('ensid not ok')
    # returned dictionary with keys as transcripID and values as the prediction values 
    return pred_dic

# print(filter_VEP_response(response, "ENSG00000258839","rs34090186"))
# print(filter_VEP_response(response, "ENSG00000185149","rs199805893"))


def add_sift_polyphen(old,new):
    '''Retrieve the information from VEP using the varID and EnsID to fill in the score 
    impact columns for SIFT and PolyPhen. As there could be more than one transcript
    for the same geneID and varID store all the values separed by white spaces'''
    with open(os.path.join(resultspath,old),"r") as myprotfile:
        with open(os.path.join(resultspath,new), 'w') as outfile:   
            myprotread = csv.reader(myprotfile, delimiter=';') 
            mywriter = csv.writer(outfile,delimiter=';') # final csv
            for myrow in myprotread:
                if myrow[0]=="Family": # write first colum of the new csv
                    mynewrow=myrow
                    mywriter.writerow(myrow)                
                    continue
                elif myrow[8]!='':
                    mynewrow=myrow
                    #check ensemblID and varID  to get the info
                    # keys=['sift_prediction','sift_score','polyphen_prediction','polyphen_score']
                    ensID=myrow[7]
                    varID=myrow[8]
                    print(varID)
                    VEPresp=request_VEP(varID)#list (long info from VEP)
                    filteredVEPresp=filter_VEP_response(VEPresp,ensID,varID)# dictionary 
                    #19,20,21,22
#                     print(varID,filteredVEPresp)
                    
                    if len(filteredVEPresp.keys())==1:
                        for transID in filteredVEPresp:#only one
                            auxDic=filteredVEPresp[transID]
                            siftData=auxDic['Sift']
                            mynewrow[20]=(transID+':'+str(siftData[0]))
                            mynewrow[19]=(transID+':'+siftData[1])
                            ppData=auxDic['Polyphen']
                            mynewrow[22]=(transID+':'+str(ppData[0]))
                            mynewrow[21]=(transID+':'+ppData[1])
                    elif len(filteredVEPresp.keys())>1:# more than one transcript
                        for transID in filteredVEPresp:
                            auxDic=filteredVEPresp[transID]
                            siftData=auxDic['Sift']
                            ppData=auxDic['Polyphen']
                            if mynewrow[19]=='?' and mynewrow[20] == '?':
                                mynewrow[20]=(transID+':'+str(siftData[0])+'   ')
                                mynewrow[19]=(transID+':'+siftData[1]+'   ')
                                mynewrow[22]=(transID+':'+str(ppData[0])+'   ')
                                mynewrow[21]=(transID+':'+ppData[1]+'   ')
                            else:
                                mynewrow[20]+=(transID+':'+str(siftData[0])+'   ')
                                mynewrow[19]+=(transID+':'+siftData[1]+'   ')
                                mynewrow[22]+=(transID+':'+str(ppData[0])+'   ')
                                mynewrow[21]+=(transID+':'+ppData[1]+'   ')
                    mywriter.writerow(mynewrow)
                    
        return print('The file', new, 'has been succesfully created.')
                    
# add_sift_polyphen("variants_ALL_GNOMAD_DISGENET.csv", "variants_ALL_GNOMAD_DISGENET_PRED.csv")                    

In [35]:
# # manage VEP responses
# first=response[0]
# for element in first:
#     print(element,':')
#     print(first[element],'\n')
# print('\n')
# sec=response[1]
# for element in first:
#     print(element,':')
#     print(sec[element],'\n')   
    
# count=0
# keys=['sift_prediction','sift_score','polyphen_prediction','polyphen_score','transcript_id']
# for element in response:
    
#     prediction_list=element['transcript_consequences']
#     for dictionary in prediction_list:
#         count+=1
#         print('Reponse element number:', count)
#         for key, value in dictionary.items():
#             if key in keys:
#                 print(key,'\t',value)
#     print('\n')
    

## Arrange variant information
1. Affecting selected positions: 23x49, 12x48, 4x39, 6x29 or 3x50 -> DONE
2. Probable functional impact: SIFT 'deleterious', PolyPhen 'Probably damaging'-> 0,1,2 DONE
3. Related with at least one disease -> DONE
4. With PDB structure information and preferably with the active and inactive states -> DONE
5. Variant in segment with 3D information. -> as we have selected the positions we know they are part of helixes 3 (TM3 3x50), 4 (TM4 4x39) and 6 (TM& 6x29) and intracelullar loop 1 (ICL1, 12x48) and extracellular loop 1 (ECL1 23x49).
6. PTM information, PTMdb***

#### Probable functional impact: SIFT 'deleterious', PolyPhen 'Probably damaging' and relation with diseases
Change the way in which we see the impacts. Set '0' if both predictors are below the scores that set the variants as 'deleterious' (SIFT) and 'probably_damaging' (PolyPhen2). Set '1' if just one predictor catalogs the variant as damaging and '2' if both predictors do.

*Note: the PolyPhen2 tag, 'possibly damaging' is not considered high impact score (approx 0.7).*


In [36]:
def split_and_add(listPredictions, dicPredictions, position):
    '''split the strings that come from the csv, colums of the predictions, by ':' and
    them to a dictionary. Used by the function 'fuse_preds' '''
    for element in listPredictions:
        if element != '':
            items=element.split(':')#[transID, predictionValue]
            if position==0:#void dictionary
                dicPredictions[items[0]]=[items[1]]
            else:    
                dicPredictions[items[0]].append(items[1])
    return None

def fuse_preds(siftScore,siftPred,ppScore,ppPred):
    '''Build a dictionary with keys as transID and values as siftScore, siftPred,
    ppScore, ppPred'''
    returnDic={}
    #sift
    predListSift=siftPred.split('   ')# transID1:'deleterious'1, transID2:'deleterious'2,...
    scoreListSift=siftScore.split('   ')# transID1:score1,...
    #poplyphen
    predListPP=ppPred.split('   ')# transID1:'deleterious'1, transID2:'deleterious'2,...
    scoreListPP=ppScore.split('   ')# transID1:score1,...
    lists=[predListSift,scoreListSift,predListPP,scoreListPP]
    index=-1
    for element in lists:
        index+=1
        split_and_add(element,returnDic,index)#split by ':'
    return returnDic


# examples:
# sp="ENST00000216629:tolerated   ENST00000553356:deleterious   ENST00000611804:tolerated   " 
# ss="ENST00000216629:0.06   ENST00000553356:0.04   ENST00000611804:0.06 "
# ppp="ENST00000216629:probably_damaging   ENST00000553356:probably_damaging   ENST00000611804:probably_damaging"
# pps="ENST00000216629:0.983   ENST00000553356:0.972   ENST00000611804:0.983   "

# sp="ENST00000555147:deleterious   ENST00000555427:deleterious   ENST00000639847:deleterious   "
# ss="ENST00000555147:0.03   ENST00000555427:0.03   ENST00000639847:0.03   "
# ppp="ENST00000555147:benign   ENST00000555427:benign   ENST00000639847:benign   "
# pps="ENST00000555147:0.248   ENST00000555427:0.342   ENST00000639847:0.248   "
# dicProva=fuse_preds(sp,ss,ppp,pps)

def get_trans(entry_name):
    '''Obtain the ENSEMBL transIDs list of the gene with the protein entry name.
    Used by the function 'clean_trans' '''
    uprot_map=uniprot_mapping('ACC+ID', 'ENSEMBL_TRS_ID', entry_name)#tab-delimited output of the mapping
    lines=uprot_map.decode().split("\n")
    tIDs=[]
    for elements in lines:
        if elements!='' and elements !='From\tTo':
            tIDs.append(elements.split("\t")[-1])
    return tIDs
#example
# print(get_trans('Q13639'))

def clean_trans(transPredDic, entryName):
    '''Only keep the transcript of the correct protein'''
    correctTrans=get_trans(entryName)# list of the two correct transcripts of the protein
#     print('Correnct transID:',correctTrans)
    correctDic={}
#     print('Intro dic:',transPredDic)
    for key, value in transPredDic.items():
        if key in correctTrans:
            correctDic[key]=value
    return correctDic
# example
# print(clean_trans(dicProva, 'Q01726'))

def set_role(myrow):
    '''Set which is the function of the AA: BS, IBS, CM depending on the variant position '''
    position=myrow[9]
    bs=['23x49'] # selected position from BS
    ibs=['12x48','4x39','6x29'] # IBS (3)
    cm_il=['3x50'] # CM-IL (also of IBS)
    if position in bs:
        function= 'LB_structure'
    elif position in ibs: 
        if position=='12x48':
            function="GProtein/ArrestinInteraction (chain B)"
        else:
            function="GProtein/ArrestinInteraction"
    else:
        function='IonicLock'
    return function

def write_newvar_line(orgRow,impactPred):
    '''New structure of the rows '''
    newrow=[]
    for i in range(0,26):
        newrow.append('')
    for i in range(0,6):
        newrow[i]=orgRow[i]
    for i in range(6,17):
        newrow[i]=orgRow[i+1]
    newrow[17]=set_role(orgRow)#role
    newrow[18]=impactPred#impact prediction:0, 1 or 2
    for i in range (19,22):#diseaseID,score, name
        newrow[i]=orgRow[i+10]
    newrow[22]=orgRow[-1]#numer of diseases
    for i in range(23,26):#foldchange, ptm, ptmtype
        newrow[i]='?'
    return newrow

def compute_impact(dictionary):
    '''Check if the variant is predicted to have impact by one or two prediction tools '''
    impacto=0
    for trasid in dictionary:
        listPrediction=dictionary[trasid]
        if listPrediction[0]=='deleterious' and listPrediction[2]=='probably_damaging':
            impacto=2#both predict impact
        elif listPrediction[0]=='deleterious' or listPrediction[2]=='probably_damaging':
            impacto=1
    return impacto


In [37]:
columNamesORG=["Family0","Name0","Short1","Uniprot ID2","Uniprot entry3","Struc4","Ligandtype5",
               "gene6","varID7","GPCRdb8","SequenceNumber9","Segment10","GPCRdbWT11","NMaa12",
               "MutationType13","Allele Count14","Allele Frequency15","Allele Number16",
               "Number of Homozygotes17","sift_word18","sift_score19","polyphen_word20",
               "polyphen_score21","GProteinInteraction22","ArrestinInteraction23",
               "ActivationPathway24","MicroSwitch25","SodiumPocket26","foldchangeMaxAbs27",
               "diseaseId28","score29","diseaseName30","Type31","PTMsite32",
               "LB_structure33","LB_fam34","related_diseases35"]

# OUT:
# "ligandType" = row 6
# "Number of Homozygotes" = row 18

# "activationPathway"=row 24
# is out because it is descrived as 'Class A activation pathway positions: '3x46','6x37','7x53' 
# and none of those positions are considered'

# "LB_fam" = row 35 (ligand binding interaction observed in a crystal structure within 
#            the same receptor family)

# REPLACED:
# "sift_word","sift_score","polyphen_word","polyphen_score"=row 19,20,21,22
# -> to 'Impact prediction', 0,1,2 (none, one, both predict impact)

# "GProteinInteraction","ArrestinInteraction","MicroSwitch","SodiumPocket","LB_structure"
# = row22,23,25,26,34
# those will be changed by 'Role': BS, IBS/ionicLock (CM),IBS(arrestin/gprotint/chainA/B)

# DOUBTS:
# "foldchangeMaxAbs"= maximum (absolute) observed foldchange in in vitro pharmacological experiments
# "PTMsite" = PTM site (yes/no) 
# "Type" = PTM type (replaced by PTMtype)


columNamesNEW=["Family","Name","Short","Uniprot ID","Uniprot entry","Struc","gene","varID",
               "GPCRdb","SequenceNumber","Segment","GPCRdbWT","NMaa","MutationType",
               "Allele Count", "Allele Frequency","Allele Number","Role","Impact Prediction",
               "diseaseId","score","diseaseName","related_diseases","foldchangeMaxAbs","PTMsite", "PTMtype"]


# build new csv with new column structure
def new_col_struc(old,new):
    with open(old,"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        with open(new,"w") as myvarfile_pred: 
            mywriter = csv.writer(myvarfile_pred, delimiter=';') 
            for myrow in myvarread:
                if myrow[0]=="Family": #first row, new names
                    mywriter.writerow(columNamesNEW)
                    continue            
                else:
                    if myrow[20]!='?' and  myrow[22]!='?':# there are impact predictors
                        #compute impact value (0,1,2)
                        print(myrow[3])
                        impact=0#default not impact
                        predDicAux=fuse_preds(myrow[20],myrow[19],myrow[22],myrow[21])#siftScore,siftPred,ppScore,ppPred
                        #now filter the transID of the ensID of the entry name
                        predDic=clean_trans(predDicAux, myrow[3])#prediction Dic and prot name
                        print(predDic)
                        impact=compute_impact(predDic)# change to 1,2
                        print(impact)
                    else:# no impact information
                        impact=None

                    mynewrow=write_newvar_line(myrow,impact)
                    mywriter.writerow(mynewrow)
    return None
                    
variants_aff_pos=os.path.join(resultspath,"variants_ALL_GNOMAD_DISGENET_PRED.csv")
variants_aff_pos_pred=os.path.join(resultspath,"filtered_var_list/vars_with_impact.csv")

# new_col_struc(variants_aff_pos,variants_aff_pos_pred)

In [7]:
def get_trans(entry_name):
    '''Obtain the ENSEMBL transIDs list of the gene with the protein entry name.
    Used by the function 'clean_trans' '''
    uprot_map=uniprot_mapping('ACC+ID', 'ENSEMBL_TRS_ID', entry_name)#tab-delimited output of the mapping
    lines=uprot_map.decode().split("\n")
    tIDs=[]
    for elements in lines:
        if elements!='' and elements !='From\tTo':
            tIDs.append(elements.split("\t")[-1])
    return tIDs
#example
# print(get_trans('Q13639'))

def read_prots(fileProt):
    listProts=[]
    count=0
    with open(os.path.join(resultspath,fileProt),"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=',') 
        for myrow in myvarread:
            if myrow[0]=="Family": #first row, new names
                continue            
            else:
                if myrow[3]=='P30988-2':
                    break
                else:
                    listProts.append(myrow[3])
    myvarfile.close()
#     print(listProts)
    return listProts

protList=read_prots(os.path.join(resultspath,'myprot_list_struc_ALL.csv'))

['P08908', 'P28222', 'P28221', 'P28566', 'P30939', 'P28223', 'P41595', 'P28335', 'Q13639', 'P47898', 'P50406', 'P34969', 'P11229', 'P08172', 'P20309', 'P08173', 'P08912', 'P35348', 'P35368', 'P25100', 'P08913', 'P18089', 'P18825', 'P08588', 'P07550', 'P13945', 'P21728', 'P14416', 'P35462', 'P21917', 'P21918', 'P35367', 'P25021', 'Q9Y5N1', 'Q9H3N8', 'Q96RJ0', 'P30556', 'P50052', 'P35414', 'P28336', 'P30550', 'P32247', 'P46663', 'P30411', 'P32238', 'P32239', 'Q16581', 'P21730', 'Q9P296', 'P25101', 'P24530', 'P21462', 'P25090', 'P25089', 'P47211', 'O43603', 'O60755', 'Q92847', 'P30968', 'Q969F8', 'Q99705', 'Q969V1', 'Q01726', 'Q01718', 'P41968', 'P32245', 'P33032', 'O43193', 'Q9HB89', 'Q9GZQ4', 'Q9GZQ6', 'Q9Y5X5', 'Q6W5P4', 'P48145', 'P48146', 'P25929', 'P49146', 'P50391', 'Q15761', 'Q99463', 'P30989', 'O95665', 'P41143', 'P41145', 'P35372', 'P41146', 'O43613', 'O43614', 'Q96P65', 'P49683', 'P25116', 'P55085', 'O00254', 'Q96RI0', 'Q9HBX9', 'Q8WXD0', 'Q9NSD7', 'Q8TDU9', 'P30872', 'P30874',

In [9]:
transProtDic={}# key: UniprotName, value:[transIDs]
for element in protList:
    transProtDic[element]=get_trans(element)

In [10]:
print(transProtDic)

{'P08908': ['ENST00000323865'], 'P28222': ['ENST00000369947'], 'P28221': ['ENST00000374619'], 'P28566': ['ENST00000305344'], 'P30939': ['ENST00000319595'], 'P28223': ['ENST00000378688', 'ENST00000542664', 'ENST00000543956'], 'P41595': ['ENST00000258400'], 'P28335': ['ENST00000276198', 'ENST00000371950', 'ENST00000371951'], 'Q13639': ['ENST00000360693', 'ENST00000362016', 'ENST00000377888', 'ENST00000517929', 'ENST00000520514', 'ENST00000521530', 'ENST00000521735', 'ENST00000522588', 'ENST00000631296'], 'P47898': ['ENST00000287907'], 'P50406': ['ENST00000289753'], 'P34969': ['ENST00000277874', 'ENST00000336152', 'ENST00000371719'], 'P11229': ['ENST00000306960', 'ENST00000543973'], 'P08172': ['ENST00000320658', 'ENST00000401861', 'ENST00000445907', 'ENST00000453373'], 'P20309': ['ENST00000255380', 'ENST00000615928'], 'P08173': ['ENST00000433765'], 'P08912': ['ENST00000383263', 'ENST00000557872'], 'P35348': ['ENST00000276393', 'ENST00000354550', 'ENST00000380572', 'ENST00000380573', 'ENST

In [13]:
setList=set()
for element in protList:
    setList.add(element)
len(setList)

285

In [12]:
len(transProtDic.keys())
len(protList)

294

In [38]:
def add_index_disbool(old, new):
    ''' Add a first column as an index and another one with T/F values to know if the variant
    is related or not with a disease'''
    columNamesNEW_with_index=["Number","Family","Name","Short","Uniprot ID","Uniprot entry","Struc","gene","varID",
               "GPCRdb","SequenceNumber","Segment","GPCRdbWT","NMaa","MutationType",
               "Allele Count", "Allele Frequency","Allele Number","Role","Impact Prediction",
                "diseaseId","score","diseaseName","related_diseases","disease"]
    
    count=0                          
    with open(os.path.join(resultspath,old),"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        with open(os.path.join(resultspath,new),"w") as myvarfile_pred: 
            mywriter = csv.writer(myvarfile_pred, delimiter=';') 
            for myrow in myvarread:
                if myrow[0]=="Family": #first row, new names
                    mywriter.writerow(columNamesNEW_with_index)
                    continue            
                else:
                    mynewrow=myrow[:-2]
                    mynewrow.insert(0, count)
                    if myrow[20]!='?':
                        mynewrow[-1]=1
                    else: 
                        mynewrow[-1]=0
                    mywriter.writerow(mynewrow)
                count+=1
# add_index_disbool('filtered_var_list/vars_with_impact.csv','filtered_var_list/vars_with_impact_index.csv')

#### PTM information using PTMdb

In [39]:
### how to? https://research.bioinformatics.udel.edu/iptmnet/about/api
# http://dbptm.mbc.nctu.edu.tw/index.php

## Variant selection
In order to select the variants to simulare we will add a total score to each of the,. It will consider:
+ Frequency, considered via *Impact Scores*: 
 + SIFT (https://sift.bii.a-star.edu.sg/www/nprot.2009.86.pdf): 'For a given protein sequence, SIFT compiles a dataset of functionally related  protein  sequences  by  searching  a  protein  database  using  the PSI-BLAST algorithm6. It then builds an alignment from the homo-logous sequences with the query sequence. In the second step of thealgorithm, SIFT scans each position in the alignment and calculates the probabilities  for  all  possible  20  amino  acids  at  that  position.  These probabilities are normalized by  the probability of the most frequent amino  acid  and  are  recorded  in  a  scaled  probability  matrix. [...]'.
 + Polyphen2 (http://genetics.bwh.harvard.edu/pph2/dokuwiki/overview): 'Elements of the matrix (profile scores) are logarithmic ratios of the likelihood of given amino acid occurring at a particular position to the likelihood of this amino acid occurring at any position (background frequency). [...]'
 So , we will consider the variants with score 2, tagged as deleterious by both impact predicting scores.

+ If they are related with a disease.

+ PDB structure.

In [40]:
def select_vars(file_vars):
    ''' Read the file where the variants with index are defined and filter the ones
    with score 2, one pdb structure, related with a disease'''
    vars_dic={}
    with open(os.path.join(resultspath,file_vars),"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        for myrow in myvarread:
            score=0
            if myrow[0]=='Number':
                continue
            else:
                if myrow[19]=='2':#impact prediction
                    score+=10
                if myrow[6]!= '':#pdb structure
                    score+=10
                if myrow[23]!= '?':#related with disease
                    score+=int(myrow[23])
            if score>19:
                newrow=myrow
                newrow.append(score)
                vars_dic[myrow[8]]=newrow#key entry name, value the row of the excel
    return vars_dic
selected_variants= select_vars('filtered_var_list/vars_with_impact_index.csv')

In [41]:
# for key, value in selected_variants.items():
#     if value[-1]>20:
#         print(value[5],value[8],value[-1])
len(selected_variants.keys() ) 

98

## GPCR selection
### Compute total score
In order to select the GPCRs to simulate we will add a total score to each GPCR. It will be computed as:

+ +1 if the allele frequency is avobe the mean on the obtained frequencies.

+ +1 If it has variants in more than one of the selected positions (we have 5 positions so here the max score is 4).

+ +0.5 If it is related with one disease.

+ Consensus impact score: +0 if it is 'Benign', +1 if it is considered 'deleterious' by one predicting tool and +2 if it is predicted as 'deleterious' by both tools.

*** We could also consider if the variant is located in a PTM but we don't know how to do it yet.

In [42]:
def first_row(rowValues):
    '''Given a row, add +1 if the allele freq is avobe the mean, +0.5 if related with a 
    disease, return the final score and also the postion'''
    scoreRow=0
    freqMean=0.00045013584139848024# obatined in variants_info_plots.ipynb
    if rowValues[23]!='?': #related with a disease +0.5
        scoreRow+=int(rowValues[23])/2
    if float(rowValues[16])>freqMean:# high frequency
        scoreRow+=1
    if rowValues[19]:
        scoreRow+=int(rowValues[19])# impact score
    position=rowValues[9]
    dataList=[scoreRow,position]
    return dataList


def total_score(old,new):
    '''Add final column with the total score for each GPCR. Print the GPCRs that 
    have an score avobe the mean + one strandard deviation (computed in 'variants_info_plots.ipynb'
    with that last column) and that have at least two states in PDB. Build a dict with the GPCRs with high total 'simulation' score 
    '''
    highImpact={}
    with open(os.path.join(resultspath,old),"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=';') 
        with open(os.path.join(resultspath,new),"w") as myvarfile_pred: 
            mywriter = csv.writer(myvarfile_pred, delimiter=';') 
            scoreDic={}
            for myrow in myvarread:
                if myrow[0]=="Number": #first row, new names
                    mynewrow=myrow
                    mynewrow.append('total_score')
                    mywriter.writerow(mynewrow)
                    continue            
                else:
                    entry=myrow[5]                    
                    mynewrow=myrow
                    
                    if entry not in scoreDic.keys(): #the entry name is new, first we consider the 1st row
                        positions=set()
                        scorePos=first_row(myrow)#score of the row, first position
                        positions.add(scorePos[1])
                        numPos=1
                        scoreDic[entry]=scorePos[0]
                    else:#the entry name has already an analyzed row
                        scorePos=first_row(myrow)
                        positions.add(scorePos[1])
                        newNumPos=len(positions)
                        if newNumPos!=numPos:#new position +1
                            scoreDic[entry]+=1
                            numPos=newNumPos
                        if newNumPos==3:# to select one to be displayed
                            print ('***',entry, newNumPos)
                        scoreDic[entry]+=scorePos[0]
                    # mean +2 sigma fo the total score (last colum of the created file)
                    #and two PDBs, computed with the df of variants_info_plots.ipynb
#                     if scoreDic[entry]>2.3+2.15*2 and myrow[6].count('&')>0:
                    if scoreDic[entry]>1 and myrow[6].count('&')>0: #mean 
                        print(entry,myrow[9],  myrow[11],scoreDic[entry])
                        highImpact[entry]=myrow
                    mynewrow.append(scoreDic[entry])#entry_name
                    mywriter.writerow(mynewrow)
                    
    return (highImpact)
     
possibleGPCRs=total_score('filtered_var_list/vars_with_impact_index.csv','filtered_var_list/vars_with_total_score.csv')

5ht1f_human 12x48 ICL1 2
5ht1f_human 12x48 ICL1 4
5ht2b_human 6x29 TM6 2
5ht2b_human 4x39 TM4 4
5ht2b_human 4x39 TM4 5
5ht5a_human 12x48 ICL1 2
*** bkrb2_human 3
*** c3ar_human 3
*** c3ar_human 3
galr2_human 23x49 ECL1 2
kissr_human 3x50 TM3 2
*** nmur1_human 3
*** nmur1_human 3
*** npsr1_human 3
oprm_human 12x48 ICL1 3.0
oprm_human 12x48 ICL1 7.0
oprm_human 3x50 TM3 8.0
rxfp1_human 6x29 TM6 2
ssr1_human 12x48 ICL1 2
ssr4_human 3x50 TM3 3
ssr4_human 3x50 TM3 5
*** ssr4_human 3
ssr4_human 6x29 TM6 7
ssr5_human 3x50 TM3 3
ssr5_human 3x50 TM3 5
*** cml1_human 3
cxcr1_human 3x50 TM3 2
cxcr1_human 12x48 ICL1 3
cxcr2_human 3x50 TM3 2
xcr1_human 3x50 TM3 2
xcr1_human 3x50 TM3 4
ackr1_human 4x39 TM4 2
ackr3_human 3x50 TM3 2
*** oxer1_human 3
*** fpr2_human 3
*** s1pr4_human 3
ptafr_human 4x39 TM4 2
*** mtr1b_human 3
aa2ar_human 3x50 TM3 2
aa2ar_human 3x50 TM3 4
aa2ar_human 4x39 TM4 6
*** p2ry6_human 3
opn5_human 23x49 ECL1 2
gpr17_human 3x50 TM3 2
gpr31_human 3x50 TM3 2
gpr31_human 3x50 TM3 4


In [43]:
print(possibleGPCRs.keys())


dict_keys(['5ht1f_human', '5ht2b_human', '5ht5a_human', 'galr2_human', 'kissr_human', 'oprm_human', 'rxfp1_human', 'ssr1_human', 'ssr4_human', 'ssr5_human', 'cxcr1_human', 'cxcr2_human', 'xcr1_human', 'ackr1_human', 'ackr3_human', 'ptafr_human', 'aa2ar_human', 'opn5_human', 'gpr17_human', 'gpr31_human', 'gpr63_human', 'gpr78_human', 'gp142_human', 'lgr5_human'])


Check if they are GPCRs are drug targets:https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5820538/ (excel downloaded in data/Studied_GPCR_vars/Targets-Drugs_table/Targets-Drugs.xlsx

In [44]:
def read_targets(file_tagets):
    ''' Read the file where the drug targets are classified and build the dictionary'''
    target_dic={'consensus':[], 'not-consensus':[], 'not-consensus*':[],'not-target':[]}
    with open(os.path.join(datapath,file_tagets),"r") as myvarfile:
        myvarread = csv.reader(myvarfile, delimiter=',') 
        for myrow in myvarread:
            if myrow[0]=='Consensus': # Confirmed drug target by all data sources
                target_dic['consensus'].append(' '+myrow[1][0:4])
            elif myrow[0]=='Not-consensus':# Confirmed drug target by not all data sources
                target_dic['not-consensus'].append(' '+myrow[1][0:4])
            elif myrow[0]=='Not-consensus, with comments': #Confirmed drug target by few data sources
                target_dic['not-consensus*'].append(' '+myrow[1][0:4])
            elif myrow[0]=='Not drug target':
                target_dic['not-target'].append(' '+myrow[1][0:4])
    return target_dic
targets=read_targets('Targets-Drugs_table/Targets-Drugs.csv')

In [45]:
targets

{'consensus': [' ADOR',
  ' ADOR',
  ' ADOR',
  ' ADOR',
  ' ADRA',
  ' ADRA',
  ' ADRA',
  ' ADRA',
  ' ADRA',
  ' ADRA',
  ' ADRB',
  ' ADRB',
  ' ADRB',
  ' AGTR',
  ' AVPR',
  ' AVPR',
  ' AVPR',
  ' BDKR',
  ' CASR',
  ' CCR5',
  ' CHRM',
  ' CHRM',
  ' CHRM',
  ' CNR1',
  ' CXCR',
  ' CYSL',
  ' DRD1',
  ' DRD2',
  ' DRD3',
  ' DRD4',
  ' DRD5',
  ' EDNR',
  ' EDNR',
  ' F2R',
  ' FSHR',
  ' GABB',
  ' GABB',
  ' GLP1',
  ' GNRH',
  ' HCRT',
  ' HCRT',
  ' HRH1',
  ' HRH2',
  ' HTR1',
  ' HTR1',
  ' HTR1',
  ' HTR1',
  ' HTR2',
  ' HTR2',
  ' HTR2',
  ' HTR4',
  ' LHCG',
  ' MTNR',
  ' MTNR',
  ' OPRD',
  ' OPRK',
  ' OPRM',
  ' OXTR',
  ' P2RY',
  ' PTGE',
  ' PTGE',
  ' PTGE',
  ' PTGE',
  ' PTGF',
  ' PTGI',
  ' S1PR',
  ' S1PR',
  ' SMO',
  ' SSTR',
  ' SSTR',
  ' SSTR',
  ' SSTR',
  ' TACR'],
 'not-consensus': [' CALC',
  ' CHRM',
  ' CHRM',
  ' CNR2',
  ' CYSL',
  ' FFAR',
  ' FPR1',
  ' GCGR',
  ' GHRH',
  ' GLP2',
  ' HCAR',
  ' HRH4',
  ' HTR1',
  ' HTR5',
  ' HTR6',
  '

In [46]:
for gpcrs in possibleGPCRs.keys():
#     print(gpcrs)
    gpcrs=gpcrs.upper()
    gpcr=gpcrs[:-6]
#     print(gpcr)
    if targets['consensus'].count(' '+gpcr)>0:
        print('Drug target', gpcr)
    elif targets['not-consensus'].count(' '+gpcr)>0 or targets['not-consensus*'].count(' '+gpcr)>0:
        print('Possible drug target', gpcr)
    elif targets['not-target'].count(' '+gpcr)>0:
        print('Not drug target', gpcr)
    else:
        print('Not in the file', gpcr)

Not in the file 5HT1F
Not in the file 5HT2B
Not in the file 5HT5A
Not in the file GALR2
Not in the file KISSR
Drug target OPRM
Not in the file RXFP1
Not in the file SSR1
Not in the file SSR4
Not in the file SSR5
Not in the file CXCR1
Not in the file CXCR2
Not drug target XCR1
Not in the file ACKR1
Not in the file ACKR3
Not in the file PTAFR
Not in the file AA2AR
Not drug target OPN5
Not in the file GPR17
Not in the file GPR31
Not in the file GPR63
Not in the file GPR78
Not in the file GP142
Not drug target LGR5
